In [1]:
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening device driver
2023-10-30 16:02:44.624 | INFO     | SiliconDriver   - Detected 4 PCI devices
2023-10-30 16:02:44.646 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 3 device_id: 0xfaca revision: 0)
2023-10-30 16:02:44.651 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 2 device_id: 0xfaca revision: 0)
2023-10-30 16:02:44.655 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 1 device_id: 0xfaca revision: 0)
2023-10-30 16:02:44.664 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 0 device_id: 0xfaca revision: 0)
2023-10-30 16:02:44.735 | INFO     | SiliconDriver   - Disable PCIE DMA
2023-10-30 16:02:44.735 | INFO     | SiliconDriver   - Disable PCIE DMA
2023-10-30 16:02:44.735 | INFO  

In [2]:
h = 31
w = 64

# Initialize tensors a and b with random values using torch

In [3]:
torch_a = torch.rand((h, w), dtype=torch.bfloat16)
torch_b = torch.rand((h, w), dtype=torch.bfloat16)

a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)

a = ttnn.to_device(a, device)
b = ttnn.to_device(b, device)

# Add tensor a and b

In [ ]:
output = a + b

# Inspect the result of the add

In [ ]:
print("Printing ttnn tensor")
print(output.shape)
print(output[:1])

# Close the device

In [ ]:
ttnn.close(device)